In [51]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import nltk
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
import numpy as np
import nltk
from sklearn.model_selection import train_test_split

%matplotlib inline

In [52]:
choni_csv = pd.read_csv('choni.csv');
choni_csv.head(10)

,tweets,quote_status,language,favorite_count,retweet_count,time,id
0,RT @chonigif: the roles have been reversed 👀\r...,False,en,0,55,2021-02-27 05:07:08,1365528872494727173
1,RT @BarchieUpdates: Rachel Talalay directed th...,False,en,0,30,2021-02-27 04:58:47,1365526773782302722
2,Stop. Shipping. Cheryl. With. Men. She. Is. A....,False,en,5,1,2021-02-27 04:57:21,1365526410257907713
3,RT @chonistyles: CHONI ARE THE MAIN STORYLINE ...,False,en,0,36,2021-02-27 04:55:52,1365526040425205762
4,RT @tcpazs: barchie and choni besties,True,en,0,3,2021-02-27 04:40:44,1365522231158333440
5,I’m gonna build choni a cute city apartment in...,False,en,0,0,2021-02-27 04:28:20,1365519111401443329
6,if choni don’t end up together by the end of t...,False,en,1,0,2021-02-27 04:25:35,1365518417755844611
7,RT @tonistopazz: when riverdale pulls a glee a...,False,en,0,3,2021-02-27 04:25:01,1365518276273582080
8,@choni_wayhaught Didn’t she film ahead?,False,en,0,0,2021-02-27 04:17:36,1365516406645796866
9,RT @vancssamorgan: you don’t understand i NEED...,False,en,0,7,2021-02-27 04:16:24,1365516106559971328


In [53]:
# Rule-based approach
posTokens = ["love", "sexy", "hot", "good", "perfect", "beautiful", "like", "sexi", "sexxy", "luv", "fav", "favorite", "riot", 
             "endgame"]
negTokens = ["beef", "choke", "fail", "barf", "hate", "yasss", "trash", "disgusting", "if i cant have", "if i want them", 
             "quit", "give up", "boring", "boring", "ugly", "dumb", "hatred", "tsk", "...", "bad"]
sentiment = []

In [54]:
for column_name in choni_csv:
    print(column_name)

tweets
quote_status
language
favorite_count
retweet_count
time
id


In [55]:
for column_name, item in choni_csv.iteritems():
    if column_name == 'tweets':
        print(item)
        print('------')

0      RT @chonigif: the roles have been reversed 👀\r...
1      RT @BarchieUpdates: Rachel Talalay directed th...
2      Stop. Shipping. Cheryl. With. Men. She. Is. A....
3      RT @chonistyles: CHONI ARE THE MAIN STORYLINE ...
4                  RT @tcpazs: barchie and choni besties
                             ...                        
697    @rcnnnie As much as I do like some of the het ...
698    @rcnnnie Give me Choni over B*rchie sex any da...
699    I'm sorry but barchie got two sex scenes durin...
700    @chonicarrington Omg yes and barchie having mo...
701    Finna pull a little nappy nap and try to dream...
Name: tweets, Length: 702, dtype: object
------


In [59]:
def sentiment_by_rules(text):
    sValue = 0
    textSplit = text.split()
    print(textSplit)
    for x in textSplit:
        if any(ele for ele in posTokens if(ele in x)):
            sValue = sValue + 1
        elif any(ele for ele in negTokens if(ele in x)):
            sValue = sValue -1
    sentiment.append(sValue)
    print(sValue)


In [60]:
for ind in choni_csv.index:
    sentiment_by_rules(choni_csv['tweets'][ind])

['RT', '@chonigif:', 'the', 'roles', 'have', 'been', 'reversed', '👀', '[2x16', '-', '5x07]', '#riverdale', '#choni', '@VanessaMorgan', '@madelainepetsch', 'https://t.co/0YiSaS3YUH']
0
['RT', '@BarchieUpdates:', 'Rachel', 'Talalay', 'directed', 'the', 'episode?', 'And', 'I’m', 'getting', 'both', 'Barchie', 'and', 'Choni', 'on', 'march', '17???', 'AAAAAAA', 'https://t.co/eq3rO8F…']
0
['Stop.', 'Shipping.', 'Cheryl.', 'With.', 'Men.', 'She.', 'Is.', 'A.', 'Lesbian.', 'https://t.co/XlVGAywHlm']
0
['RT', '@chonistyles:', 'CHONI', 'ARE', 'THE', 'MAIN', 'STORYLINE', 'FOR', '5X08', 'https://t.co/BniRoAmhOD']
0
['RT', '@tcpazs:', 'barchie', 'and', 'choni', 'besties']
0
['I’m', 'gonna', 'build', 'choni', 'a', 'cute', 'city', 'apartment', 'in', 'the', 'sims', 'I', 'think']
0
['if', 'choni', 'don’t', 'end', 'up', 'together', 'by', 'the', 'end', 'of', 'this', 'season', 'i’ll', 'riot']
1
['RT', '@tonistopazz:', 'when', 'riverdale', 'pulls', 'a', 'glee', 'and', 'has', 'choni', '&amp;', 'kangs', 'have

0
['RT', '@adoreschoni:', 'gonna', 'be', 'sick.....?', 'https://t.co/8WtVVsdp1P']
-1
['RT', '@starrytopaz:', 'i', 'just', 'think', 'it’s', 'a', 'crime', 'that', 'we’ve', 'never', 'seen', 'cheryl', 'pull', 'toni', 'onto', 'her', 'lap']
0
['RT', '@chonigif:', '"...', "you're", 'sensational."', '"...', 'we', 'need', 'you."', '[2x14', '-', '5x05]', '#riverdale', '#choni', 'https://t.co/l1GrK0sjqs']
-2
['RT', '@ruestopaz:', 'has', 'anyone', 'checked', 'on', 'that', 'person', 'who', 'tweeted', 'that', 'they', 'didnt', 'want', 'cheryl', 'to', 'cry', 'in', 's5']
0
['RT', '@sapphicvanessa:', 'hello', 'everyone', 'cheryl', 'blossom', 'wore', 'lingerie', 'to', 'go', 'see', 'her', 'ex', 'girlfriend']
0
['RT', '@Tonicheryltopaz:', 'That', 'office', 'scene', 'was', 'foreplay', 'to', 'Cheryl']
0
['RT', '@topazgotjuice:', 'I', 'WASNT', 'EXPECTING', 'TO', 'SEE', 'HER', 'FACE', 'TODAY', 'https://t.co/KFAWDKlVdJ']
0
['RT', '@choniswitches2:', 'Toni', 'really', 'had', 'a', 'year', 'old', 'master', 'plan',

In [58]:
choni_csv["Sentiment Val"] = sentiment
choni_csv.head(10)

,tweets,quote_status,language,favorite_count,retweet_count,time,id,Sentiment Val
0,RT @chonigif: the roles have been reversed 👀\r...,False,en,0,55,2021-02-27 05:07:08,1365528872494727173,0
1,RT @BarchieUpdates: Rachel Talalay directed th...,False,en,0,30,2021-02-27 04:58:47,1365526773782302722,0
2,Stop. Shipping. Cheryl. With. Men. She. Is. A....,False,en,5,1,2021-02-27 04:57:21,1365526410257907713,0
3,RT @chonistyles: CHONI ARE THE MAIN STORYLINE ...,False,en,0,36,2021-02-27 04:55:52,1365526040425205762,0
4,RT @tcpazs: barchie and choni besties,True,en,0,3,2021-02-27 04:40:44,1365522231158333440,0
5,I’m gonna build choni a cute city apartment in...,False,en,0,0,2021-02-27 04:28:20,1365519111401443329,0
6,if choni don’t end up together by the end of t...,False,en,1,0,2021-02-27 04:25:35,1365518417755844611,1
7,RT @tonistopazz: when riverdale pulls a glee a...,False,en,0,3,2021-02-27 04:25:01,1365518276273582080,0
8,@choni_wayhaught Didn’t she film ahead?,False,en,0,0,2021-02-27 04:17:36,1365516406645796866,0
9,RT @vancssamorgan: you don’t understand i NEED...,False,en,0,7,2021-02-27 04:16:24,1365516106559971328,0
